**Postgrado UAI**

Aprendizaje Reforzado para Sistemas Autónomos - [Prof. Miguel Solis](http://www.miguelsolis.info)

Sesión 02 - 09 de agosto de 2023

In [6]:
import numpy as np

In [5]:
# Ejemplo 1 - Construcción de grilla

class Mapa:

    def __init__(self,ancho,alto,inicio):
        self.ancho = ancho
        self.alto = alto
        self.fila = inicio[0]
        self.columna = inicio[1]

    def conjunto(self,recompensas,acciones):
        self.recompensas = recompensas # diccionario de posibles recompensas para cada estado (fila,columna) : r
        self.acciones = acciones # diccionario de posibles acciones para cada estado (fila,columna) : ('U','D','L','R')

    def fijar_estado(self,s):
        self.fila = s[0]
        self.columna = s[1]

    def estado_actual(self):
        return (self.fila,self.columna)

    def es_final(self,s):
        return s not in self.acciones

    def mover(self,accion):
        if accion in self.acciones[self.estado_actual()] :
            if accion == 'U':
                self.fila -= 1
            elif accion == 'D':
                self.fila += 1
            elif accion == 'L':
                self.columna -= 1
            elif accion == 'R':
                self.columna += 1
        return self.recompensas.get(self.estado_actual(),0)

    def deshacer(self,accion):
        if accion in self.acciones[self.estado_actual()] :
            if accion == 'U':
                self.fila += 1
            elif accion == 'D':
                self.fila -= 1
            elif accion == 'L':
                self.columna += 1
            elif accion == 'R':
                self.columna -= 1
        assert(self.estado_actual() in self.todos_estados())

    def fin(self):
        return self.estado_actual not in self.acciones

    def todos_estados(self):
        return set(list(self.acciones.keys()) + list(self.recompensas.keys()))

In [19]:
# Ejemplo 1 - Configuración del problema

def mapa_con_costo(costo_paso = -0.1):
    g = Mapa(4,3,(2,0)) # ancho de 4 celdas, alto de 3 celdas y posición inicial en (2,0)
    recompensas = {(0,3) : 1, (1,3):-1}
    acciones = {
        (0,0) : ('D','R'),
        (0,1) : ('L','R'),
        (0,2) : ('L','R','D'),
#        (0,3) : ('',),
        (1,0) : ('U','D'),
#        (1,1) : ('',),
        (1,2) : ('U','D','R'),
#        (1,3) : ('',),
        (2,0) : ('U','R'),
        (2,1) : ('L','R'),
        (2,2) : ('U','L','R'),
        (2,3) : ('L','U')
        }
    g.conjunto(recompensas,acciones)
    recompensas = {}
    for fila in range(g.alto):
        for columna in range(g.ancho):
            recompensas[(fila,columna)] = costo_paso
    recompensas[(0,3)] = 1
    recompensas[(1,3)] = -1
    recompensas.pop((1,1),None)
    g.conjunto(recompensas,g.acciones)
    return g

In [20]:
# Ejemplo 1 - Mostrar valores
def mostrar_valores(V,g):
    for fila in range(g.alto):
        print(20*"-")
        for columna in range(g.ancho):
            v = V.get((fila,columna),0)

            #if (v >= 0) :
            print(" %.2f|" % v, end="")
            #else :
             #   print "%.2f|"  % v,
        print("")

def mostrar_politica(P,g) :
    for fila in range(g.alto):
        print(20*"-")
        for columna in range(g.ancho):
            a = P.get((fila,columna),'')
            print(" %s |" % a, end="")
        print("")

In [21]:
# Value Iteration
repeticiones = 20
iteraciones = 0
gamma = 0.9
epsilon = 1e-3
total_acciones = ('U','D','L','R')
mapa = mapa_con_costo()
estados = mapa.todos_estados()
V = {}
for s in estados:
    V[s] = 0.0
while True:
    delta = 0
    iteraciones += 1
    for s in estados:
        if s not in mapa.acciones.keys():
            continue
        v = V[s]
        max_val = float("-inf")
        for accion in mapa.acciones[s]:
            mapa.fijar_estado(s)
            r = mapa.mover(accion)
            val = r + gamma * V[mapa.estado_actual()] # acciones deterministicas (probabilidad 1 o 0)
            if val > max_val:
                max_val = val
        V[s] = max_val
        delta = max(delta,abs(v-V[s]))

    if delta < epsilon or iteraciones >= repeticiones:
        mostrar_valores(V,mapa)
        break

--------------------
 0.62| 0.80| 1.00| 0.00|
--------------------
 0.46| 0.00| 0.80| 0.00|
--------------------
 0.31| 0.46| 0.62| 0.46|


In [6]:
# greedy
pi = {}
for s in mapa.acciones.keys():
    max_val = float("-inf")
    for accion in mapa.acciones[s]:
        mapa.fijar_estado(s)
        r = mapa.mover(accion)
        val = r + V[mapa.estado_actual()]
        if val > max_val :
            max_val = val
            max_accion = accion

    pi[s] = max_accion

mostrar_politica(pi,mapa)

--------------------
 R | R | R |  |
--------------------
 U |  | U |  |
--------------------
 U | R | U | L |
